In [1]:
!pip install -q matplotlib

In [2]:
# TODO: Change this as required
CHUNK_NUM = "01"
prefix = "shard_"
input_f = prefix + CHUNK_NUM
output_dir = 'outputs/'

output_f = output_dir  + "output_" + prefix + CHUNK_NUM + '.csv'

In [3]:
import pandas as pd
import re

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

import pickle
from typing import List

import os
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
tqdm.pandas()

In [4]:
df = pd.read_json(input_f, lines=True)

In [5]:
def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove special characters like &gt;
    text = re.sub(r'&\w+;', '', text)
    return text.lstrip('1234567890')

In [6]:
import time
import pickle

cnt = 0
total = 0

start_time = time.time()

partial_rows = []

for i in range(df.shape[0]):
  for j in range(len(df['posts'][i])):
    total += 1
    if 'perspectives' not in df['posts'][i][j] or 'com' not in df['posts'][i][j] or 'no' not in df['posts'][i][j] or 'now' not in df['posts'][i][j]:
      continue

    ps = df['posts'][i][j]['perspectives']
    c = clean_text(df['posts'][i][j]['com'])
    no = df['posts'][i][j]['no']
    ts = df['posts'][i][j]['now']

    if len(c) < 15:
      continue

    if max(ps.values()) >= 0.5:
      continue

    if 'http' in c:
      continue

    transformed_row = {k: v for k, v in ps.items()}
    transformed_row['com'] = c
    transformed_row['no'] = no
    transformed_row['ts'] = ts

    partial_rows.append(transformed_row)
    cnt += 1
    # print(ps)
    # print(c)
    # print(no)
    # print(ts)

transformed_df = pd.DataFrame(partial_rows)
transformed_df.to_csv(input_f + '_transformed.csv', index_label = 'id')

#pickle.dump(partial_rows, open(input_f + '_transformed.pkl', 'wb'))
end_time = time.time()

print(cnt)
print(total)

secs = end_time - start_time
per_entry = secs / total
print(f"For 100k entries it should take: {100000 * per_entry}")
print(f"Filtered out {100.0 * (total - cnt) / total} % of entries")


55506
199539
For 100k entries it should take: 5.392411025204845
Filtered out 72.1828815419542 % of entries


In [7]:
final_df = pd.read_csv(input_f + '_transformed.csv')
final_df

,id,TOXICITY,SEVERE_TOXICITY,INFLAMMATORY,PROFANITY,INSULT,OBSCENE,SPAM,com,no,ts
0,0,0.126195,0.083703,0.354760,0.078008,0.112733,0.093946,0.069017,he turned into literal pepe,80954178,07/14/16(Thu)07:54:40
1,1,0.079845,0.043078,0.242158,0.039846,0.058307,0.051892,0.183864,He was trying to send us a message,80954335,07/14/16(Thu)07:57:06
2,2,0.065207,0.030527,0.133405,0.031394,0.043343,0.029833,0.380979,moments like these forever make dubya Jr my fa...,80954483,07/14/16(Thu)07:59:14
3,3,0.225471,0.171812,0.168730,0.148287,0.260690,0.158239,0.381874,Fell off the wagon,80954819,07/14/16(Thu)08:05:00
4,4,0.321572,0.186607,0.283588,0.127734,0.179559,0.049037,0.042150,Go to sleep Australia.,80955590,07/14/16(Thu)08:16:11
...,...,...,...,...,...,...,...,...,...,...,...
55501,55501,0.111460,0.045015,0.279013,0.051000,0.081779,0.067192,0.122615,He&#039;s not even in the country right now. A...,81180125,07/15/16(Fri)17:27:36
55502,55502,0.137705,0.082830,0.178720,0.099004,0.136454,0.091857,0.307266,only with we meme it hard enough,81180184,07/15/16(Fri)17:27:47
55503,55503,0.188761,0.132228,0.351280,0.129384,0.190045,0.032346,0.042172,"However, this coup was most likely secular. ve...",81180509,07/15/16(Fri)17:28:59
55504,55504,0.278866,0.118437,0.167252,0.165594,0.305657,0.017850,0.237301,I thought your internet was out,81180512,07/15/16(Fri)17:29:00


In [8]:
# Load the model and tokenizer once
model_name = 'microsoft/DialoGPT-medium'
tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.padding_side = "left"
# tokenizer.pad_token = '<pad>'
model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda')

def get_dialogpt_response(comment):
    # Tokenize the input comment
    input_ids = tokenizer.encode(comment + tokenizer.eos_token, return_tensors="pt").to('cuda')

    # Generate a response
    response_ids = model.generate(input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(response_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)

    return response

In [9]:
import pandas as pd

start_idx = 0

def process_and_save(final_df, func, b_chunk_size=200):
    """Applies a function to the DataFrame in chunks and saves periodically."""

    for i in range(start_idx, len(final_df), b_chunk_size):
        chunk = final_df.iloc[i:i + b_chunk_size]
        chunk['response'] = chunk['com'].progress_apply(func)

        # Append to CSV file (mode='a' for append)
        chunk.to_csv(output_f, mode='a', header=i == 0, index=False)

process_and_save(final_df, get_dialogpt_response)

  0%|          | 0/200 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/scratch/1028977/ipykernel_2152780/774280913.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['response'] = chunk['com'].progress_apply(func)


  0%|          | 0/200 [00:00<?, ?it/s]

/scratch/1028977/ipykernel_2152780/774280913.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['response'] = chunk['com'].progress_apply(func)


  0%|          | 0/200 [00:00<?, ?it/s]

...

  0%|          | 0/200 [00:00<?, ?it/s]

/scratch/1028977/ipykernel_2152780/774280913.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['response'] = chunk['com'].progress_apply(func)


  0%|          | 0/200 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1841 > 1024). Running this sequence through the model will result in indexing errors


ValueError: Input length of input_ids is 1841, but `max_length` is set to 1000. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

In [ ]:
sanity = pd.read_csv(output_f)
sanity